Tools for building AI agents

In [12]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [ ]:
# Arxiv and wikipedia..
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun # this is looking for APIs
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper # this is useing API

In [6]:
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper= wiki_api_wrapper)
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\dp971\\Coading\\GEN_AI\\AgenticAI\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=5, lang='en', load_all_available_meta=False, doc_content_chars_max=250))

In [7]:
wiki.name

'wikipedia'

In [9]:
arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=5, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
arxiv

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=5, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250))

In [10]:
arxiv.name

'arxiv'

Now we have wikipedia and arxiv tool

In [14]:
## Custom tools[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [31]:
loader=WebBaseLoader("https://nothinginmoderation.blog/relationship-anarchy-explained-9d078f7be98d")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000026D7FAB18E0>, search_kwargs={})

now let's make this retriever a tool

In [32]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever=retriever, name="relationship_guide", description="Search any information aboutrelationship")
retriever_tool.name

'relationship_guide'

In [33]:
tools = [wiki, arxiv, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\dp971\\Coading\\GEN_AI\\AgenticAI\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=5, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=5, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='relationship_guide', description='Search any information aboutrelationship', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000026D52ACC360>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0

call the LLM


In [34]:
from langchain_groq import ChatGroq

llm = ChatGroq(model = "Llama3-8b-8192")

tools and llm is ready

now let's add prompt

In [35]:
## Prompt Template
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

Now we have llm, tools and prompts. Let's build and agent

In [36]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

Now let's execute the agent

In [37]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [ ]:
agent_executor.invoke({"input": "2502.15676"})

### React Agent

In [49]:
## Prompt Template
from langchain import hub
prompt=hub.pull("hwchase17/react")
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [51]:
from langchain.agents import create_react_agent
react_agent = create_react_agent(llm, tools, prompt)
react_agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nrelationship_guide(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever(tags=[\'FAISS\', \'OpenAIEmbeddings\'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000026D7FAB18E0>, search_kwargs={}), document_prompt: \'BasePromptTemplate\' =

In [52]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=react_agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\narxiv - A wrapper around Arxiv.org Useful for when you need to answer questions about Physics, Mathematics, Computer Science, Quantitative Biology, Quantitative Finance, Statistics, Electrical Engineering, and Economics from scientific articles on arxiv.org. Input should be a search query.\nrelationship_guide(query: \'str\', *, retriever: \'BaseRetriever\' = VectorStoreRetriever(tags=[\'FAISS\', \'OpenAIEmbeddings\'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000026D7FAB18E0>, sea

In [53]:
agent_executor.invoke({"input": "Tell me about relationship management?"})



> Entering new AgentExecutor chain...
Thought: Relationship management is a broad topic that can refer to various aspects of personal or professional relationships. It's unclear what specific aspect of relationship management the question is asking about.

Action: wikipedia
Action Input: "relationship management"

c:\Users\dp971\Coading\GEN_AI\AgenticAI\.venv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\dp971\Coading\GEN_AI\AgenticAI\.venv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Customer relationship management
Summary: Customer relationship management (CRM) is a process in which a business or another organization administers its interactions with customers, typically using data analysis to study large amounts of inforThought: The observation suggests that CRM is a process focused on managing interactions with customers. However, the question asks about relationship management in general, which could encompass a broader scope.

Action: relationship_guide
Action Input: "relationship management"The primary relationship dictates constraints for the secondary relationship(s).Relationship anarchy is effectively a non-hierarchical relationship structure. All relationship boundaries are a fluid negotiation between the whole network rather than some relationships having set limits for others beforehand. This is not to say that it’s a total free-for-all. Changing the nature of one relationship can certainly still impact another positively or negatively. But it’s 

c:\Users\dp971\Coading\GEN_AI\AgenticAI\.venv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\dp971\Coading\GEN_AI\AgenticAI\.venv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Customer relationship management
Summary: Customer relationship management (CRM) is a process in which a business or another organization administers its interactions with customers, typically using data analysis to study large amounts of inforQuestion: Tell me about relationship management?

Thought: Relationship management is a broad topic that can refer to various aspects of personal or professional relationships. It's unclear what specific aspect of relationship management the question is asking about.

Action: wikipedia
Action Input: "relationship management"

c:\Users\dp971\Coading\GEN_AI\AgenticAI\.venv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file c:\Users\dp971\Coading\GEN_AI\AgenticAI\.venv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Page: Customer relationship management
Summary: Customer relationship management (CRM) is a process in which a business or another organization administers its interactions with customers, typically using data analysis to study large amounts of inforBased on the observations, it seems that the initial question "Tell me about relationship management" is quite broad and can encompass various aspects of personal or professional relationships. The Wikipedia search revealed that CRM is a process focused on managing interactions with customers, but this may not be what the question is asking about.

The relationship guide search provided more insight into the topic, discussing relationship anarchy, non-hierarchical relationships, and hierarchical relationships in polyamory. This suggests that the question may be asking about relationship management in a more personal or emotional sense, rather than a professional or business context.

Taking this into account, I will provide a final answer t

{'input': 'Tell me about relationship management?',
 'output': 'Relationship management refers to the process of maintaining, nurturing, and navigating various types of relationships, whether personal or professional. This can include building trust, communication, conflict resolution, and conflict prevention, as well as setting boundaries and navigating power dynamics. In a personal sense, relationship management may involve managing multiple relationships, such as in a polyamorous relationship, or navigating complex emotional dynamics within a single relationship. In a professional sense, relationship management may involve building and maintaining customer relationships, managing team dynamics, or navigating conflicts within a workplace. Ultimately, effective relationship management is crucial for building strong, healthy, and fulfilling relationships in all aspects of life.'}